In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging
import os

In [ ]:
# --- CONFIGURATION ---
input_file = r'./data/taskdata_out/NORMALIZED_FIELD_DATA.csv'
output_folder = r'./data/maps_output'
os.makedirs(output_folder, exist_ok=True)

# CRS SETTINGS (Denmark UTM32N)
CRS_WGS84 = "EPSG:4326"
CRS_PROJ = "EPSG:25832" 

In [ ]:
# GRID SETTINGS
OUTPUT_RESOLUTION = 1.0  # We want a 1x1 meter final map
PROCESSING_BLOCK = 5.0   # We average raw data into 5m blocks first to remove noise/speed up Kriging

# --- 1. LOAD & REPROJECT ---
print("Loading Normalized Data...")
df = pd.read_csv(input_file)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=CRS_WGS84
)

# Reproject to Meters (UTM 32N)
gdf_meters = gdf.to_crs(CRS_PROJ)
gdf_meters['X'] = gdf_meters.geometry.x
gdf_meters['Y'] = gdf_meters.geometry.y

In [ ]:
# --- 2. KRIGING FUNCTION ---
def generate_kriging_map(field_name, crop, year, data):
    print(f"Processing: {field_name} | {crop} | {year} ({len(data)} points)")
    
    # --- STEP A: DOWNSAMPLE (NOISE REDUCTION) ---
    # Harvester data is too dense and noisy for raw Kriging.
    # We bin data into 5x5m blocks and take the mean. 
    # This acts as a 'Nugget' filter and speeds up processing 100x.
    
    data['X_Block'] = (data['X'] // PROCESSING_BLOCK) * PROCESSING_BLOCK
    data['Y_Block'] = (data['Y'] // PROCESSING_BLOCK) * PROCESSING_BLOCK
    
    # Group by spatial block
    clean_data = data.groupby(['X_Block', 'Y_Block'])['Yield_Relative_Pct'].mean().reset_index()
    
    # Convert back to arrays for PyKrige
    x_input = clean_data['X_Block'].values
    y_input = clean_data['Y_Block'].values
    z_input = clean_data['Yield_Relative_Pct'].values
    
    print(f"  -> Downsampled from {len(data)} to {len(clean_data)} points for model fitting.")
    
    if len(clean_data) < 10:
        print("  -> Not enough data for Kriging. Skipping.")
        return

    # --- STEP B: ORDINARY KRIGING ---
    # variogram_model='spherical' is standard for yield data (spatial correlation drops off at a distance)
    # verbose=False suppresses the math logs
    try:
        OK = OrdinaryKriging(
            x_input, 
            y_input, 
            z_input, 
            variogram_model='spherical',
            verbose=False,
            enable_plotting=False
        )
    except Exception as e:
        print(f"  -> Kriging Model Failed: {e}")
        return

    # --- STEP C: DEFINE 1x1 METER TARGET GRID ---
    # We define the grid based on the original extent, not the downsampled one
    x_min, x_max = data['X'].min(), data['X'].max()
    y_min, y_max = data['Y'].min(), data['Y'].max()
    
    grid_x = np.arange(x_min, x_max, OUTPUT_RESOLUTION)
    grid_y = np.arange(y_min, y_max, OUTPUT_RESOLUTION)
    
    # Execute Kriging on the fine grid
    # z_interp is the Map. sigma_squared is the Variance (Uncertainty) Map.
    z_interp, sigma_squared = OK.execute('grid', grid_x, grid_y)

    # --- STEP D: VISUALIZATION ---
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # 1. Plot the Kriged Map
    # extent=(left, right, bottom, top)
    im = ax.imshow(
        z_interp, 
        extent=(x_min, x_max, y_min, y_max), 
        origin='lower', 
        cmap='RdYlGn', 
        vmin=50, vmax=150,  # 50% to 150% Relative Yield
        alpha=0.8
    )
    
    # 2. Add Background Map (Orthophoto)
    try:
        ctx.add_basemap(
            ax, 
            crs=CRS_PROJ, 
            source=ctx.providers.Esri.WorldImagery, 
            attribution_size=6,
            zoom=17
        )
    except: pass

    # 3. Add Labels
    cbar = plt.colorbar(im, ax=ax, shrink=0.5)
    cbar.set_label('Relative Yield (%)')
    
    plt.title(f"Kriging Yield Map: {field_name}\n{crop} ({year})")
    plt.xlabel("Easting (m)")
    plt.ylabel("Northing (m)")
    
    # Save Map
    filename = f"{field_name}_{crop}_{year}_Kriging.png".replace(" ", "_").replace("/", "-")
    plt.savefig(os.path.join(output_folder, filename), dpi=150, bbox_inches='tight')
    
    # --- OPTIONAL: SAVE VARIANCE MAP (Error Map) ---
    # This shows you where the interpolation is guessing (e.g. gaps in tracks)
    # Uncomment if you want to see the error distribution
    # fig2, ax2 = plt.subplots(figsize=(10,10))
    # im2 = ax2.imshow(sigma_squared, extent=(x_min, x_max, y_min, y_max), origin='lower', cmap='Reds')
    # plt.title(f"Uncertainty Map: {field_name}")
    # plt.savefig(os.path.join(output_folder, f"{filename}_Error.png"))
    
    plt.close('all')

# --- 3. RUN LOOP ---
# Iterate through every Field/Year/Crop combination
unique_combos = gdf_meters.groupby(['FieldName', 'Year', 'Crop'])

for (field, year, crop), group in unique_combos:
    # Only map fields with substantial data
    if len(group) > 500:
        generate_kriging_map(field, crop, year, group)

print(f"Kriging Complete. Maps saved to: {output_folder}")